In [1]:
name_arg = {'name':'name','type':'String'}
weight_arg = {'name':'weight','type':'f64'}
link_arg = {'name':'link','type':'String'}
joint_arg = {'name':'joint','type':'String'}
freq_arg = {'name':'frequency','type':'f64'}
link1_arg = {'name':'link1','type':'String'}
joint1_arg = {'name':'joint1','type':'String'}
link2_arg = {'name':'link2','type':'String'}
joint2_arg = {'name':'joint2','type':'String'}

objective_data = [
    {'variant':'PositionMatch','fields':[name_arg,weight_arg,link_arg]},
    {'variant':'OrientationMatch','fields':[name_arg,weight_arg,link_arg]},
    {'variant':'PositionLiveliness','fields':[name_arg,weight_arg,link_arg,freq_arg]},
    {'variant':'OrientationLiveliness','fields':[name_arg,weight_arg,link_arg,freq_arg]},
    {'variant':'PositionMirroring','fields':[name_arg,weight_arg,link1_arg,link2_arg]},
    {'variant':'OrientationMirroring','fields':[name_arg,weight_arg,link1_arg,link2_arg]},
    {'variant':'PositionBounding','fields':[name_arg,weight_arg,link_arg]},
    {'variant':'OrientationBounding','fields':[name_arg,weight_arg,link_arg]},
    {'variant':'JointMatch','fields':[name_arg,weight_arg,joint_arg]},
    {'variant':'JointLiveliness','fields':[name_arg,weight_arg,joint_arg,freq_arg]},
    {'variant':'JointMirroring','fields':[name_arg,weight_arg,joint1_arg,joint2_arg]},
    {'variant':'JointBounding','fields':[name_arg,weight_arg,joint_arg]},
    {'variant':'JointLimits','fields':[name_arg,weight_arg]},
    {'variant':'CollisionAvoidance','fields':[name_arg,weight_arg]},
    {'variant':'VelocityMinimization','fields':[name_arg,weight_arg]},
    {'variant':'AccelerationMinimization','fields':[name_arg,weight_arg]},
    {'variant':'JerkMinimization','fields':[name_arg,weight_arg]},
    {'variant':'OriginVelocityMinimization','fields':[name_arg,weight_arg]},
    {'variant':'OriginAccelerationMinimization','fields':[name_arg,weight_arg]},
    {'variant':'OriginJerkMinimization','fields':[name_arg,weight_arg]},
    {'variant':'RelativeMotionLiveliness','fields':[name_arg,weight_arg,link1_arg,link2_arg,freq_arg]},
    {'variant':'OriginPositionLiveliness','fields':[name_arg,weight_arg,freq_arg]},
    {'variant':'OriginOrientationLiveliness','fields':[name_arg,weight_arg,freq_arg]},
    {'variant':'OriginPositionMatch','fields':[name_arg,weight_arg]},
    {'variant':'OriginOrientationMatch','fields':[name_arg,weight_arg]},
    {'variant':'Gravity','fields':[name_arg,weight_arg,link_arg]},
    {'variant':'SmoothnessMacro','fields':[name_arg,weight_arg]},
    {'variant':'DistanceMatch','fields':[name_arg,weight_arg,link1_arg,link2_arg]},
]


In [79]:
fmt_string = \
'''
#[cfg(feature = "pybindings")]
#[pyclass(name="{variant}Objective")]
#[derive(Clone,Debug)]
pub struct Py{variant}Objective({variant}Objective);

#[cfg(feature = "pybindings")]
#[pymethods]
impl Py{variant}Objective {{
    #[new]
    pub fn new({arg_string}) -> Self {{
        Py{variant}Objective({variant}Objective::new({arg_names}))
    }}
    {getters}
}}
'''

getter_string = \
'''
    #[getter]
    pub fn get_{name}(&self) -> PyResult<{type}> {{
        Ok(self.0.{name}.clone())
    }}
'''

enum_string = \
'''
#[cfg(feature = "pybindings")]
use pyo3::prelude::*;
use crate::objectives::objective::Objective;
use crate::objectives::core::base::*;
use crate::objectives::core::bounding::*;
use crate::objectives::core::matching::*;
use crate::objectives::core::mirroring::*;
use crate::objectives::liveliness::forces::*;
use crate::objectives::liveliness::perlin::*;

#[cfg(feature = "pybindings")]
#[derive(Clone,Debug,FromPyObject)]
pub enum PyObjective {{
{objective_strings}
}}

#[cfg(feature = "pybindings")]
impl IntoPy<PyObject> for PyObjective {{
    fn into_py(self, py: Python) -> PyObject {{
        match self {{
{pyinto_strings}
        }}
    }}
}}

#[cfg(feature = "pybindings")]
impl From<Objective> for PyObjective {{
    fn from(objective:Objective) -> PyObjective {{
        match objective {{
{obj2py_strings}
        }}
    }}
}}

#[cfg(feature = "pybindings")]
impl From<PyObjective> for Objective {{
    fn from(pyobjective:PyObjective) -> Objective {{
        match pyobjective {{
{py2obj_strings}
        }}
    }}
}}
'''

def obj2str(objective):
    arg_string = ','.join(['{name}:{type}'.format(**arg) for arg in objective['fields']])
    arg_names = ','.join([arg['name'] for arg in objective['fields']])
    getters = ''.join([getter_string.format(**arg) for arg in objective['fields']])
    return fmt_string.format(**objective,arg_string=arg_string,arg_names=arg_names,getters=getters)

def header():
    objective_strings = '\t'+'\n\t'.join(['{variant}(Py{variant}Objective),'.format(**objective) for objective in objective_data])
    pyinto_strings = '\t\t\t'+'\n\t\t\t'.join(['Self::{variant}(obj) => obj.into_py(py),'.format(**objective) for objective in objective_data])
    obj2py_strings = '\t\t\t'+'\n\t\t\t'.join(['Objective::{variant}(obj) => Self::{variant}(Py{variant}Objective(obj)),'.format(**objective) for objective in objective_data])
    py2obj_strings = '\t\t\t'+'\n\t\t\t'.join(['PyObjective::{variant}(obj) => Self::{variant}(obj.0),'.format(**objective) for objective in objective_data])
    return enum_string.format(objective_strings=objective_strings,pyinto_strings=pyinto_strings,obj2py_strings=obj2py_strings,py2obj_strings=py2obj_strings)


In [80]:
print(header())
for objective in objective_data:
    print(obj2str(objective))


#[cfg(feature = "pybindings")]
use pyo3::prelude::*;
use crate::objectives::objective::Objective;
use crate::objectives::core::base::*;
use crate::objectives::core::bounding::*;
use crate::objectives::core::matching::*;
use crate::objectives::core::mirroring::*;
use crate::objectives::liveliness::forces::*;
use crate::objectives::liveliness::perlin::*;

#[cfg(feature = "pybindings")]
#[derive(Clone,Debug,FromPyObject)]
pub enum PyObjective {
	PositionMatch(PyPositionMatchObjective),
	OrientationMatch(PyOrientationMatchObjective),
	PositionLiveliness(PyPositionLivelinessObjective),
	OrientationLiveliness(PyOrientationLivelinessObjective),
	PositionMirroring(PyPositionMirroringObjective),
	OrientationMirroring(PyOrientationMirroringObjective),
	PositionBounding(PyPositionBoundingObjective),
	OrientationBounding(PyOrientationBoundingObjective),
	JointMatch(PyJointMatchObjective),
	JointLiveliness(PyJointLivelinessObjective),
	JointMirroring(PyJointMirroringObjective),
	JointBounding(PyJo